In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [3]:
# download data
train_data = datasets.FashionMNIST(
  root="data",
  train=True,
  download=True,
  transform=ToTensor()
)

test_data = datasets.FashionMNIST(
  root="data",
  train=False,
  download=True,
  transform=ToTensor()
)

In [4]:
batch_size = 64

train_dataloader = DataLoader(train_data, batch_size=batch_size) # convert into iterable of batches
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(X.shape)
  print(y.shape, y.dtype)
  break

torch.Size([64, 1, 28, 28])
torch.Size([64]) torch.int64


In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
# optimizing
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediciton error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
 

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309043 [    0/60000]
loss: 2.290317 [ 6400/60000]
loss: 2.272482 [12800/60000]
loss: 2.267389 [19200/60000]
loss: 2.244006 [25600/60000]
loss: 2.222526 [32000/60000]
loss: 2.232144 [38400/60000]
loss: 2.199768 [44800/60000]
loss: 2.198438 [51200/60000]
loss: 2.163111 [57600/60000]
Test Error: 
 Accuracy: 45.0%, Avg loss: 2.154619 

Epoch 2
-------------------------------
loss: 2.173591 [    0/60000]
loss: 2.155401 [ 6400/60000]
loss: 2.095979 [12800/60000]
loss: 2.111155 [19200/60000]
loss: 2.063006 [25600/60000]
loss: 2.004355 [32000/60000]
loss: 2.040375 [38400/60000]
loss: 1.960314 [44800/60000]
loss: 1.962134 [51200/60000]
loss: 1.896447 [57600/60000]
Test Error: 
 Accuracy: 51.2%, Avg loss: 1.884420 

Epoch 3
-------------------------------
loss: 1.925824 [    0/60000]
loss: 1.891317 [ 6400/60000]
loss: 1.763362 [12800/60000]
loss: 1.811741 [19200/60000]
loss: 1.705322 [25600/60000]
loss: 1.649265 [32000/60000]
loss: 1.690148 [38400/

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [13]:
pred # getting the logits

tensor([[-1.8405, -2.8090, -0.6886, -1.5514, -0.8741,  2.2998, -0.9152,  2.4816,
          1.7218,  2.9086]])

In [16]:
random_data = torch.rand(3, 28, 28) # creating a random set
# two ways to get the same value
print(random_data.shape)
print(random_data.size())

torch.Size([3, 28, 28])
torch.Size([3, 28, 28])


In [17]:
flatten = nn.Flatten() # instance to flatten the array
flat_image = flatten(random_data)
print(flat_image.size()) # note that the first dimension remains

torch.Size([3, 784])


In [18]:
layer1 = nn.Linear(in_features=28*28, out_features=20) # input is image flatten and output is 20 neurons
hidden1 = layer1(flat_image) # forward pass through the layer
print(hidden1.size())

torch.Size([3, 20])


In [19]:
print(f"Before ReLU: {hidden1}\n\n") # result of each activation per image
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}") # result after activation per image

Before ReLU: tensor([[-0.4218, -0.1658, -0.2595,  0.0882,  0.0466, -0.6728, -0.4905,  0.5081,
          0.3618,  0.4115, -0.2237,  0.0749,  0.0951,  0.1120,  0.1686,  0.3804,
          0.2788, -0.3914, -0.1942, -0.6738],
        [-0.4927, -0.3630, -0.1684, -0.1227,  0.0605, -0.4330, -0.7126,  0.6651,
         -0.0563,  0.2220,  0.0334,  0.3570,  0.1054,  0.1901, -0.1289,  0.0338,
         -0.0379, -0.1796, -0.2330, -0.4424],
        [-0.4462, -0.6690, -0.0901, -0.5295,  0.1529, -0.4276, -0.5698,  0.5504,
         -0.2445,  0.4684, -0.2727,  0.1057, -0.1550,  0.1633, -0.0614,  0.3705,
          0.0146, -0.4489, -0.0071, -0.6275]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0882, 0.0466, 0.0000, 0.0000, 0.5081, 0.3618,
         0.4115, 0.0000, 0.0749, 0.0951, 0.1120, 0.1686, 0.3804, 0.2788, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0605, 0.0000, 0.0000, 0.6651, 0.0000,
         0.2220, 0.0334, 0.3570, 0.1054, 0.1901, 0.00

In [48]:
# grab the parameters of the hidden layer
params = {}
for b in layer1.named_parameters():
  params[b[0]] = b[1]
params['weight']

Parameter containing:
tensor([[ 0.0317,  0.0195, -0.0051,  ...,  0.0006,  0.0085, -0.0003],
        [-0.0121,  0.0285,  0.0175,  ..., -0.0224, -0.0051,  0.0344],
        [-0.0126,  0.0217,  0.0297,  ...,  0.0101, -0.0119, -0.0034],
        ...,
        [-0.0260,  0.0091, -0.0129,  ..., -0.0232,  0.0162,  0.0063],
        [ 0.0253, -0.0185,  0.0096,  ..., -0.0223, -0.0236,  0.0088],
        [ 0.0159,  0.0293, -0.0324,  ..., -0.0099, -0.0067, -0.0131]],
       requires_grad=True)